In [43]:
import json
from random import shuffle

In [44]:
with open('data/data_with_middle_layer.json') as json_posFile:
    posData = json.load(json_posFile)

In [45]:
print(len(posData))
print(posData[0])
x = posData[0]
len(x['leaf_categories'])

55326
{'title': 'Finally, WTO to Open Debate on Patents and Low-Cost Drugs', 'url': 'https://www.business-humanrights.org/finally-wto-to-open-debate-on-patents-and-low-cost-drugs', 'body': "The much-awaited debate on how international patent laws affect developing countries' access to medications begins Wednesday, Jun 20, at the World Trade Organisation (WTO), and the countries, corporations and civil society groups involved are putting the final touches on their arguments.", 'companies': [], 'leaf_categories': ['Pharmaceutical', 'Health: General (including workplace health & safety)', 'Access to medicines', 'Intellectual property', 'WTO'], 'middle_categories': ['Other', 'Intl. finance/trade orgs.', 'Health']}


5

In [46]:
import re
from w3lib.html import remove_tags
def clense(text, space_replacer = ' ', to_lower = True, remove_punc = True):
    # remove HTML comments first as suggested in https://stackoverflow.com/questions/28208186/how-to-remove-html-comments-using-regex-in-python
    text = re.sub("(<!--.*?-->)", "", text, flags=re.DOTALL)
    text = remove_tags(text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)   #remove non-ascii characters
    text = text.replace("&amp;", "and")
    text = text.replace("&", "and")
    text.strip()
    text.rstrip()
    text = text.replace("\r\n", "")
    text = text.replace("\n", "")
    text = text.replace("\"", "")
    if to_lower:
        text = text.lower()

    if remove_punc:
        # from https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
        text = re.sub(r'[^\w\s]', '', text)   #remove punctuation marks and non-word
        text = text.replace(",", "")

    text = re.sub(' +', space_replacer, text)
    #if  all(ord(char) < 128 for char in text) == False:
    #    text = ''
    ''.join(i for i in text if ord(i)<128)
    return text

In [47]:
# one label for each body
new_posData = []
for x in posData:
    if x['body'] is not None:
        if len(x['body']) < 5: # if body is empty
            continue
        each_data = {}
        each_data['body'] = x['body']
        unitemp = each_data['body'].encode('utf8').decode('utf8')
        each_data['body'] = clense(unitemp,to_lower=False,remove_punc=False).strip()
        each_data['category'] = []
        for y in range (len(x['leaf_categories'])):
            each_data['category'].append(x['leaf_categories'][y])
        new_posData.append(each_data)

In [48]:
print(new_posData[0])
print(new_posData[1])
print(new_posData[2])

{'body': "The much-awaited debate on how international patent laws affect developing countries' access to medications begins Wednesday, Jun 20, at the World Trade Organisation (WTO), and the countries, corporations and civil society groups involved are putting the final touches on their arguments.", 'category': ['Pharmaceutical', 'Health: General (including workplace health & safety)', 'Access to medicines', 'Intellectual property', 'WTO']}
{'body': 'A vaccine is our last, best hope of stopping the epidemic. Seth Berkley is trying to deliver the dream. If Seth Berkley finds a vaccine, he hopes to guarantee instant worldwide access to it.', 'category': ['Pharmaceutical', 'Health: General (including workplace health & safety)', 'Access to medicines', 'HIV/AIDS']}
{'body': "Program to Expand on South African Partnership Model in Cooperation With United Nations' AIDS Program and World Health Organization", 'category': ['Pharmaceutical', 'Health: General (including workplace health & safety

In [49]:
shuffle(new_posData)

In [50]:
# single data trial
print(new_posData[0])
y = new_posData[0]['category'][1]
print(y)
label = ''
cat = ''
for i in y:
    if i == ' ':
        cat = cat + '_'
    else :
        cat = cat + i
label = label + '__label__' + cat + '__  '
print(cat) 
print(label)
label = label + '__label__' + cat + '__  '
print(label)
print(label + new_posData[0]['body'])

{'body': "Without doubt the appointment of Tidjane Thiam is a major breakthrough for black Britons in business. It is a powerful indicator that things are changing, and that in some cases, and in certain companies, talented black people can make it to the top. But we shouldn't leaping to conclusions Thiam...works in a city where 30% are racial minorities. So having one man in this position highlights inequality rather than equality...Across Britain's biggest institutions, the fact is there has been little real change of culture, and the minority faces are just black and brown dots, bobbing about in a sea of white...[refers to Prudential, BT]", 'category': ['United Kingdom', 'Finance & banking', 'Insurance', 'CEOs', 'Discrimination/diversity: General', 'Racial & ethnic groups']}
Finance & banking
Finance_&_banking
__label__Finance_&_banking__  
__label__Finance_&_banking__  __label__Finance_&_banking__  
__label__Finance_&_banking__  __label__Finance_&_banking__  Without doubt the appoi

In [51]:
f = open("leaf_bhr.txt","w+")
for x in new_posData:
    label = ""
    body = x['body'].strip()
    for y in x['category']:
        cat = ''
        for i in y:
            if i == ' ':
                cat = cat + '_'
            else :
                cat = cat + i
        label = label + '__label__' + cat.strip() + '__  '        
    f.write(label + body + '\n')
f.close()